<a href="https://colab.research.google.com/github/alfredqbit/datasciencecoursera/blob/master/sepulvedaADDS-8515-5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Canonical Correlation Analysis and Multivariate Regression on the Linnerud Dataset

This notebook implements Canonical Correlation Analysis (CCA) and Multivariate Multiple Regression (MVR)
on the Linnerud exercise dataset, following the methodology described in the report.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_linnerud
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import CCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.pipeline import Pipeline

%pip install statsmodels --quiet
import statsmodels.api as sm
from statsmodels.multivariate.manova import MANOVA

sns.set(style="whitegrid")

FIG_DIR = "figures"
os.makedirs(FIG_DIR, exist_ok=True)

Helper functions: correlation matrix, VIFs, and canonical loadings and redundancy

In [ ]:
def col_corr_matrix(A, B):
    """Correlation matrix between columns of A and B."""
    A0 = A - A.mean(axis=0)
    B0 = B - B.mean(axis=0)
    cov = A0.T @ B0 / (A.shape[0] - 1)
    sdA = A0.std(axis=0, ddof=1)
    sdB = B0.std(axis=0, ddof=1)
    return cov / np.outer(sdA, sdB)

def redundancy_indices(loadings, rho_sq):
    """
    Redundancy indices for one set given loadings and squared canonical corrs.
    loadings: (variables x components) correlations with canonical variates
    rho_sq : array of squared canonical correlations
    """
    var_explained = (loadings ** 2).mean(axis=0)
    return var_explained * rho_sq

def compute_vif(X_df):
    vifs = {}
    for col in X_df.columns:
        X_other = X_df.drop(columns=[col])
        y = X_df[col]
        model = LinearRegression().fit(X_other, y)
        r2 = model.score(X_other, y)
        vif = 1.0 / (1.0 - r2)
        vifs[col] = vif
    return pd.Series(vifs, name="VIF")

Load dataset and basic EDA (Step 1)

In [ ]:
linnerud = load_linnerud()

X_ex = pd.DataFrame(linnerud.data, columns=linnerud.feature_names)        # Chins, Situps, Jumps
Y_phys = pd.DataFrame(linnerud.target, columns=linnerud.target_names)     # Weight, Waist, Pulse

print("Exercise variables (X):")
display(X_ex.head())

print("\nPhysiological variables (Y):")
display(Y_phys.head())

print("\nX info:")
print(X_ex.info())

print("\nY info:")
print(Y_phys.info())

print("\nSummary statistics for X:")
display(X_ex.describe())

print("\nSummary statistics for Y:")
display(Y_phys.describe())

print("\nMissing values in X:")
print(X_ex.isna().sum())
print("\nMissing values in Y:")
print(Y_phys.isna().sum())


Correlation heatmap and simple outlier check

In [ ]:
# Combine for correlation inspection
df_all = pd.concat([X_ex, Y_phys], axis=1)
corr = df_all.corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm",
            annot_kws={"size": 9}, cbar_kws={"label": "Correlation"})
plt.title("Correlation Matrix: Exercise and Physiological Variables")
plt.tight_layout()
plt.savefig(os.path.join(FIG_DIR, "linnerud_correlation_matrix.png"), dpi=300)
plt.show()

z_scores = (df_all - df_all.mean()) / df_all.std(ddof=1)
outlier_mask = (z_scores.abs() > 3)
print("Potential outliers (|z| > 3) per variable:")
print(outlier_mask.sum())

Standardization (Step 2)

In [ ]:
# Standardize each block separately
scaler_X = StandardScaler()
scaler_Y = StandardScaler()

X_scaled = scaler_X.fit_transform(X_ex)
Y_scaled = scaler_Y.fit_transform(Y_phys)

X_scaled_df = pd.DataFrame(X_scaled, columns=X_ex.columns)
Y_scaled_df = pd.DataFrame(Y_scaled, columns=Y_phys.columns)

X_scaled_df.head(), Y_scaled_df.head()

Canonical Correlation Analysis (Step 2 CCA)

Note: CCA naturally takes two inputs (X, Y), so forcing it into a single sklearn Pipeline is awkward. We keep it as a clean, modular block using the standardized matrices.

In [ ]:
# CCA with up to min(p, q) = 3 components
cca = CCA(n_components=3)
cca.fit(X_scaled, Y_scaled)

X_c, Y_c = cca.transform(X_scaled, Y_scaled)  # canonical variates

canonical_corrs = np.array([
    np.corrcoef(X_c[:, k], Y_c[:, k])[0, 1]
    for k in range(3)
])

print("Canonical correlations:")
for k, rho in enumerate(canonical_corrs, start=1):
    print(f"  rho_{k} = {rho:.3f}")

# Canonical loadings: correlations of original vars with canonical variates
load_XU = col_corr_matrix(X_scaled, X_c)
load_YV = col_corr_matrix(Y_scaled, Y_c)

load_XU_df = pd.DataFrame(load_XU, index=X_ex.columns,
                          columns=[f"u{k}" for k in range(1, 4)])
load_YV_df = pd.DataFrame(load_YV, index=Y_phys.columns,
                          columns=[f"v{k}" for k in range(1, 4)])

print("\nCanonical loadings: Exercise variables on U")
display(load_XU_df)

print("\nCanonical loadings: Physiological variables on V")
display(load_YV_df)

rho_sq = canonical_corrs ** 2
red_Y_given_X = redundancy_indices(load_YV, rho_sq)
red_X_given_Y = redundancy_indices(load_XU, rho_sq)

print("\nRedundancy indices (Y | X):", np.round(red_Y_given_X, 3))
print("Sum redundancy Y | X:", red_Y_given_X.sum())
print("Redundancy indices (X | Y):", np.round(red_X_given_Y, 3))
print("Sum redundancy X | Y:", red_X_given_Y.sum())

CCA scatterplots (canonical variates)

In [ ]:
# Scatterplot for first canonical pair
plt.figure(figsize=(6, 5))
plt.scatter(X_c[:, 0], Y_c[:, 0])
plt.axhline(0, color="gray", linewidth=0.8)
plt.axvline(0, color="gray", linewidth=0.8)
plt.xlabel("u1 (exercise canonical variate)")
plt.ylabel("v1 (physiological canonical variate)")
plt.title("CCA: First Canonical Variates (u1 vs v1)")
plt.tight_layout()
plt.savefig(os.path.join(FIG_DIR, "cca_u1_v1_scatter.png"), dpi=300)
plt.show()

plt.figure(figsize=(6, 5))
plt.scatter(X_c[:, 1], Y_c[:, 1])
plt.axhline(0, color="gray", linewidth=0.8)
plt.axvline(0, color="gray", linewidth=0.8)
plt.xlabel("u2 (exercise canonical variate)")
plt.ylabel("v2 (physiological canonical variate)")
plt.title("CCA: Second Canonical Variates (u2 vs v2)")
plt.tight_layout()
plt.savefig(os.path.join(FIG_DIR, "cca_u2_v2_scatter.png"), dpi=300)
plt.show()

VIF for predictors

In [ ]:
vif_series = compute_vif(X_ex)
print("Variance Inflation Factors (VIF):")
display(vif_series)

Optimized pipeline for Multivariate Regression: fit and metrics (Step 3 MVR)

In [ ]:

# Pipeline: Standardize X -> LinearRegression (multi-output)
mvr_pipeline = Pipeline([
    ("scaler", StandardScaler()),       # scales predictors X_ex
    ("reg",    LinearRegression())
])

mvr_pipeline.fit(X_ex, Y_phys)

# Predictions
Y_hat = pd.DataFrame(mvr_pipeline.predict(X_ex), columns=Y_phys.columns)

# Per-response R^2 and RMSE
r2_each = r2_score(Y_phys, Y_hat, multioutput="raw_values")
mse_each = mean_squared_error(Y_phys, Y_hat, multioutput="raw_values")
rmse_each = np.sqrt(mse_each)

perf = pd.DataFrame({
    "Response": Y_phys.columns,
    "R2": r2_each,
    "RMSE": rmse_each
})

print("In-sample multivariate regression performance (pipeline):")
display(perf)

Residual diagnostics (normality, homoscedasticity)

In [ ]:
residuals = Y_phys - Y_hat

for col in Y_phys.columns:
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))

    # Residual vs fitted
    axes[0].scatter(Y_hat[col], residuals[col])
    axes[0].axhline(0, color="gray", linewidth=0.8)
    axes[0].set_xlabel("Fitted values")
    axes[0].set_ylabel("Residuals")
    axes[0].set_title(f"{col}: Residuals vs Fitted")

    # Q-Q plot
    sm.qqplot(residuals[col], line="45", ax=axes[1])
    axes[1].set_title(f"{col}: Q-Q Plot")

    fig.suptitle(f"Diagnostics for {col}")
    fig.tight_layout()
    fig.savefig(os.path.join(FIG_DIR, f"diagnostics_{col.lower()}.png"), dpi=300)
    plt.show()

MANOVA for multivariate significance

In [ ]:
# Build a single DataFrame for MANOVA
df_reg = pd.concat([Y_phys, X_ex], axis=1)
formula = "Weight + Waist + Pulse ~ Chins + Situps + Jumps"

maov = MANOVA.from_formula(formula, data=df_reg)
print(maov.mv_test())